In [5]:
from kaggle.api.kaggle_api_extended import KaggleApi
from pyspark.sql import SparkSession
from datetime import date

#create API connection and authenticate
api = KaggleApi()
api.authenticate()

#download dataset 
api.dataset_download_file('deepcontractor/monkeypox-dataset-daily-updated','Daily_Country_Wise_Confirmed_Cases.csv')
api.dataset_download_file('deepcontractor/monkeypox-dataset-daily-updated','Monkey_Pox_Cases_Worldwide.csv')

#create spark session
spark = SparkSession.builder.getOrCreate()

#create spark df
daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)

def date_check():
    today = date.today()
    return str(today)

def column_grab():
    try:
        daily_df = daily_df.select(daily_df['Country'],daily_df['2022-07-01'])
        return daily_df
    except:
        pass

(daily_df[-1]).show(1)


TypeError: 'Column' object is not callable

In [ ]:
from airflow import DAG
from airflow.decorators import dag, task

#create our tasks
@task
def api_pull():
    """
    pulls the updated file from Kaggle
    """
    api = KaggleApi()
    api.authenticate()
    #download dataset 
    api.dataset_download_file('deepcontractor/monkeypox-dataset-daily-updated','Daily_Country_Wise_Confirmed_Cases.csv')
    #create spark session
    spark = SparkSession.builder.getOrCreate()


@task
def main():
    """
    main.py, update table with new data, do transformations
    """

@task
def load():
    """
    load to bigqquery
    """

#set dag attributes
@dag(
    schedule_interval='@daily',
    start_date=datetime.utcnow(),
    catchup=False,
)
#function that calls all of our tasks to create dag
def monkeypox_dag():


![image](DAG.png)

In [46]:
# daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)
#pivot


def column_grab():
    daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)
    daily_df = daily_df.select(daily_df['Country'],daily_df['2022-07-01'])
    return daily_df


daily_df = column_grab()

daily_df.show(10)

dataset_df = spark.read.csv('Monkey_Pox_Cases_Worldwide.csv', header=True)

dataset_df = dataset_df.join(daily_df,['Country'],how='left')

dataset_df.withColumn('Confirmed_Cases', dataset_df['Confirmed_Cases'] + dataset_df['2022-07-01']).drop(dataset_df['2022-07-01']).show(10)

+-------------+----------+
|      Country|2022-07-01|
+-------------+----------+
|      England|         0|
|     Portugal|         0|
|        Spain|         0|
|United States|        64|
|      Germany|        85|
|      Belgium|         0|
|       Sweden|         0|
|        Italy|        33|
|       Canada|         0|
|       France|         0|
+-------------+----------+
only showing top 10 rows

+-------------+---------------+---------------+------------+------------------+-----------------+
|      Country|Confirmed_Cases|Suspected_Cases|Hospitalized|Travel_History_Yes|Travel_History_No|
+-------------+---------------+---------------+------------+------------------+-----------------+
|      England|         1185.0|            0.0|         5.0|               2.0|              7.0|
|     Portugal|          402.0|            0.0|         0.0|               0.0|             34.0|
|        Spain|         1196.0|            0.0|        11.0|               2.0|              0.0|
|United 

In [78]:
import quinn


dataset_df = spark.read.csv('Monkey_Pox_Cases_Worldwide.csv', header=True)
daily_df = spark.read.csv('Daily_Country_Wise_Confirmed_Cases.csv', header=True)

# total_df = dataset_df.select(dataset_df['Country'], dataset_df['Confirmed_Cases'])

# dataset_df = total_df.join(daily_df,['Country'],how='left')

def dashes_to_underscores(s):
    return s.replace("-", "_")

dataset_df = dataset_df.transform(quinn.with_columns_renamed(dashes_to_underscores))

dataset_df = daily_df.transform(quinn.with_columns_renamed(dashes_to_underscores))

dataset_df.groupBy("Country").pivot("Country")

pandas_DF = dataset_df.toPandas()

pandas_DF.set_index('Country', inplace=True)
transpose_df = pandas_DF.transpose()
transpose_df.rename(columns = {'Country':'Date'})

transpose_df.reset_index(inplace=True)

spark_df=spark.createDataFrame(transpose_df)
spark_df = spark_df.withColumnRenamed('index', 'Date')

spark_df.show()




+----------+-------+--------+-----+-------------+-------+-------+------+-----+------+------+---------+-----------+-----------+------+-------+-------+--------+--------------------+--------+--------------+----------------+-----+---------+-------+-------+------+-----+------+-------+---------+-------+------+------+------+-----+------+---------+-------+-----+-------+-------+----------+-----+------+---------+-------+------------+-----------+--------+-------+--------+------+----+-------+-----------+------+
|      Date|England|Portugal|Spain|United States|Germany|Belgium|Sweden|Italy|Canada|France|Australia|Netherlands|Switzerland|Israel|Denmark|Austria|Scotland|United Arab Emirates|Slovenia|Czech Republic|Northern Ireland|Wales|Argentina|Finland|Ireland|Mexico|Malta|Norway|Hungary|Gibraltar|Morocco|Latvia|Greece|Brazil|Ghana|Poland|Venezuela|Romania|Benin|Iceland|Georgia|Luxembourg|Chile|Serbia|Singapore|Lebanon|South Africa|South Korea|Colombia|Croatia|Bulgaria|Taiwan|Peru|Estonia|Puerto Ric